# Statistics Explained  articles and matches with OECD's Glossary

## Objective: to build a common vocabulary and construct "profiles" of the terms of this vocabulary

### Installation instructions
*    Download the notebook as "raw" file and save it with extension .ipynb (cut the .txt extension which is added)
*    Install the necessary libraries from your jupyter command prompt. These, together with the versions used, are:
    *    pyodbc==4.0.32
    *    spacy==3.2.1 
    *    pandas==1.3.5
    *    nltk==3.6.5
*   Launch the notebook and put your own credentials for access to the Virtuoso database in the call to pyodbc.connect() in the chunk with title "Connect to the database"    


### The first time download the Spacy's library. Comment-out in subsequent runs

* Note that we are using Spacy's "large" library.

In [1]:
import re
import pandas as pd
import spacy
import sys
from collections import Counter

## Run to install the language library, then comment-out
!{sys.executable} -m spacy download en_core_web_lg

nlp = spacy.load('en_core_web_lg')
nlp.max_length = 1500000
print('Finished loading.')


[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
Finished loading.


In [2]:
from datetime import datetime

def file_name(pre,ext):
    current_time = datetime.now() 
    return pre + '_'+ str(current_time.month)+ '_' + str(current_time.day) + \
                 '_' + str(current_time.hour)+ '_' + str(current_time.minute)  +'.'+ext

### Connect to the database

In [3]:
import pyodbc
c = pyodbc.connect('DSN=Virtuoso All;DBA=ESTAT;UID=xxxxx;PWD=xxxxx')
cursor = c.cursor()

In [4]:
import re
#import unicodedata as ud

def clean(x, quotes=True):
    if pd.isnull(x): return x  
    x = x.strip()
    
    ## make letter-question mark-letter -> letter-quote-space-letter !!! but NOT in the lists of URLs!!!
    if quotes:
        x = re.sub(r'([A-Za-z])\?([A-Za-z])','\\1\' \\2',x) 
    
    ## make letter-question mark-space lower case letter letter-quote-space letter
    x = re.sub(r'([A-Za-z])\? ([a-z])','\\1\' \\2',x) 

    ## delete ,000 commas in numbers    
    x = re.sub(r'\b(\d+),(\d+)\b','\\1\\2',x) ## CORRECTED
    
    ## delete  000 spaces in numbers
    x = re.sub(r'\b(\d+) (\d+)\b','\\1\\2',x) ## CORRECTED
    
    ## remove more than one spaces
    x = re.sub(r' +', ' ',x)
    
    ## remove start and end spaces
    x = re.sub(r'^ +| +$', '',x,flags=re.MULTILINE) 
    
    ## space-comma -> comma
    x = re.sub(r' \,',',',x)
    
    ## space-dot -> dot
    x = re.sub(r' \.','.',x)
    
    x = re.sub(r'â.{2}',"'",x) ### !!! NEW: single quotes are read as: âXX
    
    #x = x.encode('latin1').decode('utf-8') ## â\x80\x99
    #x = ud.normalize('NFKD',x).encode('ascii', 'ignore').decode()
    
    return x

### Statistics explained articles

* IDs, titles from dat_link_info, with resource_information_id=1, i.e. Eurostat (see ESTAT.V1.mod_resource_information) and matching IDs from dat_article.
* Carry out data cleansing on titles.


In [5]:
SQLCommand = """SELECT id, title 
                FROM ESTAT.V1.dat_link_info 
                WHERE resource_information_id=1 AND id IN (SELECT id FROM ESTAT.V1.dat_article) """

SE_df = pd.read_sql(SQLCommand,c)

SE_df['title'] = SE_df['title'].apply(clean)
SE_df.head(5)


,id,title
0,7,Accidents at work statistics
1,13,National accounts and GDP
2,16,Railway safety statistics in the EU
3,17,Railway freight transport statistics
4,18,Railway passenger transport statistics - quart...


### Add paragraphs titles and contents

* From dat_article_paragraph with abstract=0 (i.e. "no").
* Match article_id from dat_article_paragraph with id from dat_article.
* Carry out data cleansing on titles and paragraph contents.

In [6]:
SQLCommand = """SELECT article_id, title, content 
                FROM ESTAT.V1.dat_article_paragraph
                WHERE abstract=0 AND article_id IN (SELECT id FROM ESTAT.V1.dat_article) """

add_content = pd.read_sql(SQLCommand,c)
add_content['title'] = add_content['title'].apply(clean)
add_content['content'] = add_content['content'].apply(clean)
add_content

,article_id,title,content
0,2905,Absences from work sharply increase in first h...,Absences from work recorded unprecedented high...
1,2905,Absences: 9.5 % of employment in Q4 2019 and 1...,The article's next figure (Figure 4) compares ...
2,2905,Higher share of absences from work among women...,"Considering all four quarters of 2020, the sha..."
3,2905,Absences from work due to own illness or disab...,"From Q4 2019 to Q4 2020, the number of people ..."
4,2905,Absences from work due to holidays,"Expressed as a share of employed people, absen..."
...,...,...,...
3854,10539,General presentation and definition,Scope of asylum statistics and Dublin statisti...
3855,10539,Methodological aspects in asylum statistics,Annual aggregate of the number of asylum appli...
3856,10539,Methodological aspects in Dublin statistics,Asymmetries For most of the collected Dublin s...
3857,10539,What questions can or cannot be answered with ...,How many asylum seekers are entering EU Member...


### Aggregate above paragraph titles and contents  from SE articles paragraphs by article id

* Create a column _raw content_ which gathers all paragraph titles and contents in one text per article.

In [7]:
add_content_grouped = add_content.groupby(['article_id'])[['title','content']].aggregate(lambda x: list(x))
add_content_grouped.reset_index(drop=False, inplace=True)
for i in range(len(add_content_grouped)):
    add_content_grouped.loc[i,'raw content'] = ''
    for (a,b) in zip(add_content_grouped.loc[i,'title'],add_content_grouped.loc[i,'content']):
        add_content_grouped.loc[i,'raw content'] += ' '+a + ' ' + b
add_content_grouped = add_content_grouped[['article_id','raw content']]    

add_content_grouped

,article_id,raw content
0,7,"Number of accidents In 2018, there were 3.1 m..."
1,13,Developments for GDP in the EU-27: growth sin...
2,16,Fall in the number of railway accidents 9 % f...
3,17,Downturn for EU transport performance in 2019...
4,18,Rail passenger transport performance continue...
...,...,...
860,10456,Problem After successfully identifying and jo...
861,10470,"Problem In France, there was significant room..."
862,10506,General overview Nine PEEIs concern short-ter...
863,10531,What are administrative sources? The term 'ad...


### Merge raw content of SE articles with main file

* Also, add title to definition.

In [8]:
SE_df = pd.merge(SE_df,add_content_grouped,left_on='id',right_on='article_id',how='inner')
SE_df.drop(['article_id'],axis=1,inplace=True)

SE_df['raw content'] = SE_df['title'] +'. '+SE_df['raw content']

SE_df.head(5)

,id,title,raw content
0,7,Accidents at work statistics,Accidents at work statistics. Number of accid...
1,13,National accounts and GDP,National accounts and GDP. Developments for G...
2,16,Railway safety statistics in the EU,Railway safety statistics in the EU. Fall in ...
3,17,Railway freight transport statistics,Railway freight transport statistics. Downtur...
4,18,Railway passenger transport statistics - quart...,Railway passenger transport statistics - quart...


### Lemmatize 'raw content'

* NLTK seems to be better than Spacy in lemmatization. Convert to lower-case before.

In [9]:
import nltk

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

SE_df['raw content'] = SE_df['raw content'].apply(lambda x: x.lower())
SE_df['raw content']= SE_df['raw content'].apply(lemmatize_text)
SE_df['raw content']= [' '.join(map(str, l)) for l in SE_df['raw content']]
SE_df['raw content'] = SE_df['raw content'].apply(lambda x: x.upper())
SE_df


,id,title,raw content
0,7,Accidents at work statistics,ACCIDENT AT WORK STATISTICS. NUMBER OF ACCIDEN...
1,13,National accounts and GDP,NATIONAL ACCOUNT AND GDP. DEVELOPMENT FOR GDP ...
2,16,Railway safety statistics in the EU,RAILWAY SAFETY STATISTIC IN THE EU. FALL IN TH...
3,17,Railway freight transport statistics,RAILWAY FREIGHT TRANSPORT STATISTICS. DOWNTURN...
4,18,Railway passenger transport statistics - quart...,RAILWAY PASSENGER TRANSPORT STATISTIC - QUARTE...
...,...,...,...
860,10456,"Merging statistics and geospatial information,...","MERGING STATISTIC AND GEOSPATIAL INFORMATION, ..."
861,10470,"Merging statistics and geospatial information,...","MERGING STATISTIC AND GEOSPATIAL INFORMATION, ..."
862,10506,Methods for compiling PEEIs in short-term busi...,METHOD FOR COMPILING PEEIS IN SHORT-TERM BUSIN...
863,10531,Building the System of National Accounts - adm...,BUILDING THE SYSTEM OF NATIONAL ACCOUNT - ADMI...


### OECD - Glossary of Statistical Terms
https://stats.oecd.org/glossary/alpha.asp

* Scrape terms and lemmatize.

In [10]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

url = "https://stats.oecd.org/glossary/alpha.asp"
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')
text = soup.get_text()
    
rows = soup.find_all('tr')
str_cells = str(rows)
cleantext = BeautifulSoup(str_cells, "lxml").get_text()
#print(cleantext)

list_rows = []
for row in rows:
    cells = row.find_all('a')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '',str_cells))
    list_rows.append(clean2)
#print(clean2)
#type(clean2)

df = pd.DataFrame(list_rows)
df.head(10)
df[0]=df[0].apply(lambda x: re.sub(r'\[',' ',x))
df[0]=df[0].apply(lambda x: re.sub(r'\]',' ',x))
df1 = df[0].str.split(',', expand=True)
df_t = df1.T
df_t=df_t[[22]]
df_t = df_t.rename(columns={22: 'term'})
nan_value = float("NaN")

df_t.replace(" ", nan_value, inplace=True)

df_t.dropna(subset = ["term"], inplace=True)
df_t.replace(" ", nan_value, inplace=True)
df_t.insert(0, 'id', range(len(df_t)))
df_t.reset_index(inplace=True)
df_t.drop(columns=['index'],inplace=True)
df_t.head()

df_t['lemmatized_term']= df_t['term'].apply(lambda x: x.lower())
df_t['lemmatized_term']= df_t['lemmatized_term'].apply(lemmatize_text)
df_t['lemmatized_term']= [' '.join(map(str, l)) for l in df_t['lemmatized_term']]
df_t['lemmatized_term']= df_t['lemmatized_term'].apply(lambda x: x.upper())
df_t

,id,term,lemmatized_term
0,0,A posteriori audit,A POSTERIORI AUDIT
1,1,A priori audit,A PRIORI AUDIT
2,2,A programme language (APL),A PROGRAMME LANGUAGE (APL)
3,3,Abatement,ABATEMENT
4,4,Abatement cost,ABATEMENT COST
...,...,...,...
7074,7074,Zero-coupon / deep discount bond,ZERO-COUPON / DEEP DISCOUNT BOND
7075,7075,Zero-coupon bonds,ZERO-COUPON BOND
7076,7076,Zones,ZONE
7077,7077,Zoning,ZONING


### Prepare Spacy's PhraseMatcher by building a custom vocabulary from OECD's Glossary (from column 'lemmatized_term')

In [11]:
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)
terms = df_t['lemmatized_term'].values.tolist()
# Only run nlp.make_doc to speed things up
patterns = [nlp.make_doc(text) for text in terms]
matcher.add("TerminologyList", patterns)

### Apply PhraseMatcher

* Collect results per SE article ('doc_id') in a dataframe 'res'. Ignore matches with 2 words or less.
* Depending on length of match: columns '3-Phrases', '4-Phrases', '5-and-above-Phrases'. These will contain dictionaries with the matched lemmatized terms and their counts, in descending order of counts.
* Column 'Terms' has a dictionary with the corresponding **original terms** in OECD's Glossary and their counts in the matches.

In [12]:
res = pd.DataFrame(index=range(len(SE_df)))
res['3-Phrases']=[[] for i in range(len(SE_df))]
res['4-Phrases']=[[] for i in range(len(SE_df))]
res['5-and-above-Phrases']=[[] for i in range(len(SE_df))]
res['Terms']=[dict() for i in range(len(SE_df))]
docs=nlp.pipe(SE_df['raw content'])
for (i,doc) in enumerate(docs):
    print(i)
    for sent in doc.sents:
        matches = matcher(sent)
        for match_id, start, end in matches:
            span = doc[start:end]
            n_words = len(span.text.split(' '))
            if n_words >= 3:
                doc_id = SE_df.loc[i,'id']
                idx = df_t.index[df_t['lemmatized_term'].str.contains(span.text,regex=False)].tolist()
                print(i,SE_df.loc[i,'title'],len(sent.text),'>',n_words,span.text,idx)
                res.loc[i,'doc_id']=doc_id
                for elem in df_t.loc[idx,'term'].values.tolist():
                    if elem in res.loc[i,'Terms'].keys():
                        res.loc[i,'Terms'][elem] +=1
                    else:
                        res.loc[i,'Terms'][elem] =1
                #res.loc[i,'Terms'].append(concepts_df.loc[idx,'term'].values.tolist())
                if n_words == 3:
                    res.loc[i,'3-Phrases'].append(span.text)
                elif n_words == 4:
                    res.loc[i,'4-Phrases'].append(span.text)
                else:
                    res.loc[i,'5-and-above-Phrases'].append(span.text)
                    
                    


0
0 Accidents at work statistics 324 > 4 HEALTH AND SOCIAL WORK [1988, 3114]
1
1 National accounts and GDP 190 > 3 RATE OF CHANGE [247, 5346, 5937, 6640]
1 National accounts and GDP 142 > 3 RATE OF CHANGE [247, 5346, 5937, 6640]
1 National accounts and GDP 254 > 3 RATE OF CHANGE [247, 5346, 5937, 6640]
1 National accounts and GDP 206 > 3 RATE OF CHANGE [247, 5346, 5937, 6640]
1 National accounts and GDP 143 > 3 RATE OF CHANGE [247, 5346, 5937, 6640]
1 National accounts and GDP 231 > 3 RATE OF CHANGE [247, 5346, 5937, 6640]
1 National accounts and GDP 206 > 3 ANNUAL GROWTH RATE [242, 248]
1 National accounts and GDP 206 > 3 RATE OF CHANGE [247, 5346, 5937, 6640]
1 National accounts and GDP 314 > 3 RATE OF CHANGE [247, 5346, 5937, 6640]
1 National accounts and GDP 173 > 3 RATE OF CHANGE [247, 5346, 5937, 6640]
1 National accounts and GDP 162 > 3 RATE OF CHANGE [247, 5346, 5937, 6640]
1 National accounts and GDP 162 > 3 PURCHASING POWER STANDARD [5219]
1 National accounts and GDP 175 > 3 

33
33 International trade in goods 148 > 3 ANNUAL GROWTH RATE [242, 248]
33 International trade in goods 169 > 3 ANNUAL GROWTH RATE [242, 248]
34
34 Material flow accounts and resource productivity 128 > 4 GROSS DOMESTIC PRODUCT (GDP) [2812, 2813, 2814, 2815, 2816, 2817, 2818, 2819, 4298, 4957]
34 Material flow accounts and resource productivity 128 > 4 DOMESTIC MATERIAL CONSUMPTION (DMC) [1713]
34 Material flow accounts and resource productivity 219 > 3 GROSS CAPITAL FORMATION [2806]
35
35 Digital economy and society statistics - enterprises 212 > 4 ELECTRONIC DATA INTERCHANGE (EDI) [1932]
36
37
37 Healthy life years statistics 133 > 3 QUALITY OF LIFE [5263]
38
39
40
40 Children at risk of poverty or social exclusion 785 > 3 TYPE OF HOUSEHOLD [6662]
40 Children at risk of poverty or social exclusion 785 > 3 LEVEL OF EDUCATION [3628, 4261, 4262, 4956, 5409]
40 Children at risk of poverty or social exclusion 785 > 3 LEVEL OF EDUCATION [3628, 4261, 4262, 4956, 5409]
40 Children at risk o

57 Structure of government debt 233 > 3 GOVERNMENT GROSS DEBT [2646, 2651, 2771]
57 Structure of government debt 304 > 3 GOVERNMENT GROSS DEBT [2646, 2651, 2771]
57 Structure of government debt 178 > 3 GOVERNMENT GROSS DEBT [2646, 2651, 2771]
57 Structure of government debt 245 > 3 GOVERNMENT GROSS DEBT [2646, 2651, 2771]
57 Structure of government debt 223 > 3 GOVERNMENT GROSS DEBT [2646, 2651, 2771]
57 Structure of government debt 175 > 3 GOVERNMENT GROSS DEBT [2646, 2651, 2771]
57 Structure of government debt 177 > 3 GOVERNMENT GROSS DEBT [2646, 2651, 2771]
57 Structure of government debt 179 > 3 GOVERNMENT GROSS DEBT [2646, 2651, 2771]
58
59
59 Healthcare resource statistics - beds 189 > 3 PSYCHIATRIC CARE BED [5185]
59 Healthcare resource statistics - beds 116 > 3 PSYCHIATRIC CARE BED [5185]
59 Healthcare resource statistics - beds 428 > 3 PSYCHIATRIC CARE BED [5185]
59 Healthcare resource statistics - beds 267 > 3 PSYCHIATRIC CARE BED [5185]
59 Healthcare resource statistics - be

98 Educational attainment statistics 149 > 3 LEVEL OF EDUCATION [3628, 4261, 4262, 4956, 5409]
98 Educational attainment statistics 232 > 3 LEVEL OF EDUCATION [3628, 4261, 4262, 4956, 5409]
99
99 Statistics on young people neither in employment nor in education or training 167 > 3 LEVEL OF EDUCATION [3628, 4261, 4262, 4956, 5409]
99 Statistics on young people neither in employment nor in education or training 238 > 3 LEVEL OF EDUCATION [3628, 4261, 4262, 4956, 5409]
99 Statistics on young people neither in employment nor in education or training 279 > 3 LEVEL OF EDUCATION [3628, 4261, 4262, 4956, 5409]
99 Statistics on young people neither in employment nor in education or training 279 > 3 LEVEL OF EDUCATION [3628, 4261, 4262, 4956, 5409]
99 Statistics on young people neither in employment nor in education or training 279 > 3 LEVEL OF EDUCATION [3628, 4261, 4262, 4956, 5409]
99 Statistics on young people neither in employment nor in education or training 255 > 3 LEVEL OF EDUCATION [362

103 Tax revenue statistics 148 > 3 ACTUAL SOCIAL CONTRIBUTION [76, 1969]
104
104 Tourism statistics 396 > 3 BALANCE OF PAYMENT [382, 384, 385, 386, 388, 390, 392, 394, 396, 398, 400, 402, 404, 406, 959, 1317, 6403, 6407]
105
105 Chapter 5 259 > 3 PLACE OF RESIDENCE [4896]
105 Chapter 5 261 > 3 PLACE OF RESIDENCE [4896]
106
106 Wages and labour costs 126 > 3 PURCHASING POWER STANDARD [5219]
107
107 Overweight and obesity - BMI statistics 193 > 3 LEVEL OF EDUCATION [3628, 4261, 4262, 4956, 5409]
107 Overweight and obesity - BMI statistics 305 > 3 LEVEL OF EDUCATION [3628, 4261, 4262, 4956, 5409]
108
109
110
110 Tourism industries - economic analysis 196 > 5 VALUE ADDED AT FACTOR COST [2841, 6828]
110 Tourism industries - economic analysis 187 > 5 VALUE ADDED AT FACTOR COST [2841, 6828]
110 Tourism industries - economic analysis 194 > 5 VALUE ADDED AT FACTOR COST [2841, 6828]
111
111 Healthcare expenditure statistics 187 > 4 GROSS DOMESTIC PRODUCT (GDP) [2812, 2813, 2814, 2815, 2816, 2817

...

731 City statistics ' education and training 477 > 3 LEVEL OF EDUCATION [3628, 4261, 4262, 4956, 5409]
731 City statistics ' education and training 262 > 3 LEVEL OF EDUCATION [3628, 4261, 4262, 4956, 5409]
732
733
733 Building the System of National Accounts - volume measures 272 > 4 CONSUMPTION OF FIXED CAPITAL [1133]
733 Building the System of National Accounts - volume measures 434 > 4 GOOD AND SERVICE ACCOUNT [2727]
733 Building the System of National Accounts - volume measures 103 > 3 PAASCHE PRICE INDEX [2693, 3716, 4682]
733 Building the System of National Accounts - volume measures 103 > 3 LASPEYRES VOLUME INDEX [3595]
733 Building the System of National Accounts - volume measures 244 > 3 LASPEYRES VOLUME INDEX [3595]
733 Building the System of National Accounts - volume measures 89 > 3 LASPEYRES PRICE INDEX [2690, 3593, 3715]
733 Building the System of National Accounts - volume measures 89 > 3 PAASCHE VOLUME INDEX [4684]
733 Building the System of National Accounts - volume m

753
754
754 Living conditions in Europe - introduction 190 > 4 GROSS DOMESTIC PRODUCT (GDP) [2812, 2813, 2814, 2815, 2816, 2817, 2818, 2819, 4298, 4957]
755
755 MEETS programme 444 > 3 FOREIGN DIRECT INVESTMENT [2506, 2507]
756
756 MEETS programme - framework for business-related statistics 176 > 5 LOCAL KIND OF ACTIVITY UNIT [3701]
756 MEETS programme - framework for business-related statistics 112 > 5 LOCAL KIND OF ACTIVITY UNIT [3701]
756 MEETS programme - framework for business-related statistics 381 > 3 FOREIGN DIRECT INVESTMENT [2506, 2507]
757
757 Merging statistics and geospatial information, 2012 projects - Hungary 52 > 4 NATIONAL STATISTICAL OFFICE (NSO) [4161]
758
758 Merging statistics and geospatial information, 2012 projects - Malta 251 > 4 NATIONAL STATISTICAL OFFICE (NSO) [4161]
759
759 Merging statistics and geospatial information, 2012 projects - Poland 358 > 3 PLACE OF WORK [2449, 4285, 4901]
759 Merging statistics and geospatial information, 2012 projects - Poland 3

789 Merging statistics and geospatial information, 2014 projects - Portugal 197 > 3 CONSUMER PRICE INDEX [1124, 1125, 2873]
790
790 Statistics in development cooperation - coordination 224 > 3 MILLENNIUM DEVELOPMENT GOAL [3973, 3974]
790 Statistics in development cooperation - coordination 224 > 3 MILLENNIUM DEVELOPMENT GOAL [3973, 3974]
790 Statistics in development cooperation - coordination 434 > 4 INTERNATIONAL DEVELOPMENT ASSOCIATION (IDA) [3344]
791
791 Statistics in development cooperation - data availability 422 > 3 CONSUMER PRICE INDEX [1124, 1125, 2873]
791 Statistics in development cooperation - data availability 422 > 3 HOUSEHOLD BUDGET SURVEY [2979]
791 Statistics in development cooperation - data availability 170 > 4 NATIONAL STATISTICAL SYSTEM (NSS) [4162]
791 Statistics in development cooperation - data availability 107 > 4 NATIONAL STATISTICAL INSTITUTE (NSI) [4160]
791 Statistics in development cooperation - data availability 39 > 3 RATE OF CHANGE [247, 5346, 5937, 66

807 Statistics in development cooperation - advocacy 385 > 3 MILLENNIUM DEVELOPMENT GOAL [3973, 3974]
808
808 Statistics in development cooperation - improving statistical capacity 294 > 3 MILLENNIUM DEVELOPMENT GOAL [3973, 3974]
808 Statistics in development cooperation - improving statistical capacity 344 > 4 NATIONAL STATISTICAL SYSTEM (NSS) [4162]
809
809 Statistics in development cooperation - EU support to partner countries 95 > 4 NATIONAL STATISTICAL SYSTEM (NSS) [4162]
809 Statistics in development cooperation - EU support to partner countries 252 > 3 SECTOR WIDE APPROACH [5802]
810
811
811 Statistics in development cooperation - development indicators 289 > 4 NATIONAL STATISTICAL SYSTEM (NSS) [4162]
812
813
813 Merging statistics and geospatial information, 2013 projects - Italy 377 > 3 PLACE OF RESIDENCE [4896]
813 Merging statistics and geospatial information, 2013 projects - Italy 84 > 3 LOSS OF INFORMATION [3745]
814
815
816
816 Merging statistics and geospatial informatio

853 Disability statistics background - Labour force survey - proxy analysis 242 > 3 LEVEL OF EDUCATION [3628, 4261, 4262, 4956, 5409]
854
854 Disability statistics background - Labour force survey - non-response analysis 189 > 3 LEVEL OF EDUCATION [3628, 4261, 4262, 4956, 5409]
854 Disability statistics background - Labour force survey - non-response analysis 303 > 3 LEVEL OF EDUCATION [3628, 4261, 4262, 4956, 5409]
854 Disability statistics background - Labour force survey - non-response analysis 194 > 3 LEVEL OF EDUCATION [3628, 4261, 4262, 4956, 5409]
854 Disability statistics background - Labour force survey - non-response analysis 286 > 3 LEVEL OF EDUCATION [3628, 4261, 4262, 4956, 5409]
855
855 Continuing Vocational Training Survey (CVTS) methodology 178 > 4 HEALTH AND SOCIAL WORK [1988, 3114]
856
856 City statistics ' economic aspects 168 > 4 INTERNATIONAL LABOUR ORGANISATION (ILO) [3351]
856 City statistics ' economic aspects 296 > 3 ECONOMICALLY ACTIVE POPULATION [1851]
856 Ci

In [13]:
res['3-Phrases']= res['3-Phrases'].apply(lambda x: dict(Counter(x).most_common()))
res['4-Phrases']= res['4-Phrases'].apply(lambda x: dict(Counter(x).most_common()))
res['5-and-above-Phrases']= res['5-and-above-Phrases'].apply(lambda x: dict(Counter(x).most_common()))



In [14]:
res=pd.merge(SE_df[['id','title']],res,left_on='id',right_on='doc_id',how='left')
res.drop(columns=['doc_id'],inplace=True)
res

,id,title,3-Phrases,4-Phrases,5-and-above-Phrases,Terms
0,7,Accidents at work statistics,{},{'HEALTH AND SOCIAL WORK': 1},{},"{' health and social work': 1, ' Incomes of he..."
1,13,National accounts and GDP,"{'RATE OF CHANGE': 15, 'GROSS VALUE ADDED': 6,...",{'HEALTH AND SOCIAL WORK': 3},"{'TAX ON PRODUCTION AND IMPORT': 2, 'EXTERNAL ...",{' Annualised growth rate (annualised rate of ...
2,16,Railway safety statistics in the EU,NaN,NaN,NaN,NaN
3,17,Railway freight transport statistics,NaN,NaN,NaN,NaN
4,18,Railway passenger transport statistics - quart...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
860,10456,"Merging statistics and geospatial information,...",NaN,NaN,NaN,NaN
861,10470,"Merging statistics and geospatial information,...",NaN,NaN,NaN,NaN
862,10506,Methods for compiling PEEIs in short-term busi...,"{'PRODUCER PRICE INDEX': 3, 'CONSUMER PRICE IN...",{},{},"{' Input producer price indices': 3, ' Output ..."
863,10531,Building the System of National Accounts - adm...,"{'GROSS VALUE ADDED': 4, 'BALANCE OF PAYMENT':...","{'CONSUMPTION OF FIXED CAPITAL': 2, 'VALUE ADD...",{'FINANCIAL INTERMEDIATION SERVICE INDIRECTLY ...,"{' Deductible value added tax (VAT)': 1, ' Inv..."


In [15]:
outfile = file_name('Phrase_Matcher_SE_OECD','xlsx')
res.to_excel(outfile)
